In [25]:
#Detect Faces

# Import the necessary libraries
import cv2
import os

# Set the path to the folder containing the images
image_folder_path = r"//"

# Set the cascades path
cascades_path = r"//"

# Create a cascade object for face detection
face_cascade = cv2.CascadeClassifier(os.path.join(cascades_path, "haarcascade_frontalface_default.xml"))

# Iterate over the images in the folder
for image_name in os.listdir(image_folder_path):
    # Load the image
    image = cv2.imread(os.path.join(image_folder_path, image_name))
    
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use the cascade object to detect faces in the image
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.3, minNeighbors=5)
    
    # Check if any faces were detected
    if len(faces) > 0:
        # Loop through the detected faces
        for (x, y, w, h) in faces:
            # Draw a rectangle around the face
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Create a resizable window
        cv2.namedWindow("Faces", cv2.WINDOW_NORMAL)
        
        # Resize the window
        cv2.resizeWindow("Faces", 1080, 720)
        
        # Show the image
        cv2.imshow("Faces", image)
        cv2.waitKey(0)

# Close all windows
cv2.destroyAllWindows()


In [26]:
import cv2
import numpy as np

# Load the cascade classifier for detecting faces
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load the face recognition model
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Read the images in the folder
images = []
for file_name in os.listdir('images/'):
    image = cv2.imread('images/' + file_name)
    images.append(image)

# Group the images by their faces
face_groups = []

# Detect faces in each image and add them to the training set
train_x = []
train_y = []
label = 0
for image in images:
    faces = face_cascade.detectMultiScale(image, 1.3, 5)

    # If no faces are detected, skip this image
    if len(faces) == 0:
        continue

    # For each face, add it to the training set
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        train_x.append(face)
        train_y.append(label)
        label += 1

# Convert the list of labels to a numpy array
train_y = np.array(train_y)

# Convert the images to grayscale
train_x = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in train_x]

# Train the face recognition model on the training set
recognizer.train(train_x, train_y)

# Group the images by their faces
face_groups = []

# Convert the images to grayscale
images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]

# Group the images by their faces
for image in images:
    faces = face_cascade.detectMultiScale(image, 1.3, 5)

    # If no faces are detected, skip this image
    if len(faces) == 0:
        continue

    # For each face, predict its label using the face recognition model
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        label, confidence = recognizer.predict(face)

        # If this is the first face in the group, create a new group
        if len(face_groups) == 0 or label not in face_groups[0][1]:
            face_groups.append([label, face])
        else:
            # Check if the face belongs to an existing group
            added = False
            for group in face_groups:
                # If the face belongs to this group, add the image to the group
                if label == group[0]:
                    group.append(face)
                    added = True
                    break

# Print the number of groups
print(f'Number of groups: {len(face_groups)}')

# # Print the images in each group
# for group in face_groups:
#     print('Group:')
#     for image in group[1:]:
#         cv2.imshow('Image', image)
#         cv2.waitKey(0)


Number of groups: 1116


In [27]:
# Create a new folder for each face group
for group in face_groups:
    label = group[0]
    os.makedirs(f'face-groups/{label}', exist_ok=True)

# Save the images in each group to the corresponding folder
for group in face_groups:
    label = group[0]
    for image in group[1:]:
        cv2.imwrite(f'face-groups/{label}/{label}.jpg', image)
